In [368]:
import nltk
from nltk import bigrams, trigrams
from nltk.tokenize import word_tokenize
from nltk.corpus import brown as brown
from nltk.stem import WordNetLemmatizer
from collections import defaultdict, Counter
from nltk import NaiveBayesClassifier
import random
import numpy
import math
# nltk.download('punkt')

In [369]:
nltk.download('brown')

sentences = nltk.corpus.brown.sents()

print(sentences)


[['The', 'Fulton', 'County', 'Grand', 'Jury', 'said', 'Friday', 'an', 'investigation', 'of', "Atlanta's", 'recent', 'primary', 'election', 'produced', '``', 'no', 'evidence', "''", 'that', 'any', 'irregularities', 'took', 'place', '.'], ['The', 'jury', 'further', 'said', 'in', 'term-end', 'presentments', 'that', 'the', 'City', 'Executive', 'Committee', ',', 'which', 'had', 'over-all', 'charge', 'of', 'the', 'election', ',', '``', 'deserves', 'the', 'praise', 'and', 'thanks', 'of', 'the', 'City', 'of', 'Atlanta', "''", 'for', 'the', 'manner', 'in', 'which', 'the', 'election', 'was', 'conducted', '.'], ...]


[nltk_data] Downloading package brown to /root/nltk_data...
[nltk_data]   Package brown is already up-to-date!


In [370]:
# TODO: Import and preprocess your dataset (tokenization, normalization, etc.)
nltk.download('brown') #Download brown corpus
sentences = nltk.corpus.brown.sents()


def start_index(x, n):
  return (x - 1) * n

def end_index(x , n):
  return x * n - 1

x = 5 # Group E
n = 10000 # num sentences to extract
S = start_index(x, n)
E = end_index(x, n)

ALPHA = 0.00000001 # for laplace smoothing

print(n)
print(f"Start: {S}")
print(f"End: {E}")
print(E - S + 1) # num sentences we are processing
sentences = sentences[S: E + 1]

# Function to get genre from a given fileid
def get_genre(fileid):
    return nltk.corpus.brown.categories(fileid)[0]
print(sentences[0])
genres = [get_genre(fileid) for fileid in nltk.corpus.brown.fileids() for sentence in nltk.corpus.brown.sents(fileids=fileid)]
genres = genres[S: E + 1]
print(numpy.unique(genres)) #print possible genres in our given range ['adventure' 'fiction' 'mystery' 'science_fiction']

[nltk_data] Downloading package brown to /root/nltk_data...
[nltk_data]   Package brown is already up-to-date!


10000
Start: 40000
End: 49999
10000
['The', 'door', 'was', 'answered', 'by', 'a', 'slender', 'man', 'in', 'his', 'sixties', '--', 'straight-backed', ',', 'somewhat', 'clerical', 'in', 'manner', ',', 'wearing', 'rimless', 'glasses', '.']
['adventure' 'fiction' 'mystery' 'science_fiction']


In [371]:
nltk.download("wordnet")

# (ths)
def preprocess_data(sentences, lemmatize=False):
    # TODO: Add preprocessing steps (tokenization, normalization, etc.)
    processed_sentences = []
    genres = set()
    for sentence in sentences:
        # Join tokens to form a string sentence
        sentence_str = ' '.join(sentence)
        # Tokenization and normalization
        tokens = word_tokenize(sentence_str.lower())  # Convert to lowercase for normalization
        # Lemmatization (optional)
        if lemmatize:
          for token in tokens:
            lemmatizer = WordNetLemmatizer()
            tokens = [lemmatizer.lemmatize(token) for token in tokens]
        # prepend and append start/end symbols
        tokens.insert(0, '<s>')
        tokens.append('</s>')
        # add tokenized sentence to list of preprocessed sentences
        processed_sentences.append(tokens)
        # print(tokens)
    return processed_sentences
# Prepare the data
 # (ths)
# sentences = nltk.corpus.brown.sents() #Let's start small with 15 sentences first
processed_sentences = preprocess_data(sentences, True)

print("length of processed_sentences:", len(processed_sentences))
# store sentences and genres as pairs
sentences_and_genres = zip(processed_sentences, genres)

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


length of processed_sentences: 10000


In [372]:
from nltk.probability import FreqDist
# Initialize models
model = defaultdict(lambda: defaultdict(lambda: 0))
# stores each trigram with a specific genre {trigram: {adventure: 5, fiction: 2}}
genres_model = defaultdict(lambda: defaultdict(lambda: 0))


# TODO: Build a trigram model (count occurrences of trigrams in the corpus)
# FreqDist of all words in our range:
fdist = FreqDist() # replaced tcd, FYI -Noah

for sentence, genre in zip(processed_sentences, genres):
  for token in sentence:
    fdist[token.lower()] += 1
  for w1, w2, w3 in trigrams(sentence, pad_right=True, pad_left=True):
    model[(w1, w2)][w3] += 1
    genres_model[(w1, w2, w3)][genre] += 1

num_tokens = len(fdist)

# (ths) <- I moved it into a function -Noah
# Print the trigram model just to see the outputs
def print_model(model):
  for w1_w2 in model:
    print(f"Current: {w1_w2}")
    for w3, probability in model[w1_w2].items():
      print(f"Next word: {w3}, Probability: {probability:.3f}")
    print()


def print_genres(model):
  for trigram in model:
    print("Trigram: ", trigram)
    for genre, count in model[trigram].items():
      print(f"Genre: {genre}, Count: {count}")

# print_genres(genres_model)
# print_model(model)

In [373]:
# TODO: Transform the counts to probabilities (MLE)
for w1_w2 in model:
    total_count = float(sum(model[w1_w2].values())) # total num of trigrams starting with w1, w2
    for w3 in model[w1_w2]:
        model[w1_w2][w3] /= total_count # replace count w/ p(w3 | w1, w2)

for trigram in genres_model:
    total_count = float(sum(genres_model[trigram].values()))
    for genre in genres_model[trigram]:
      genres_model[trigram][genre] /= total_count

# Calculate the occurrence of rare words -- check number of rare words, remember these words would have probability near zero, causing issues
def tri_count(model, trigram): # Count(X, Y, Z)
    bigram = trigram[:2]
    last_element = trigram[2]
    if bigram in model and last_element in model[bigram]:
        return model[bigram][last_element]
    else:
        return 0

def bi_count(model, bigram): # Count(X, Y)
    count = 0
    if bigram in model:
        for last in model[bigram]:
            count += model[bigram][last]
    return count

def cond_prob(model, word, bigram): # P(Z | X, Y) # NOTE: You have to pass word as tuple w/ comma: `("sarah",)`
    return tri_count(model, bigram + word) / bi_count(model, bigram)


In [380]:
# TODO: Implement a smoothing technique -- Don't use +1 approach, go with Laplace smoothing but with delta function, check instructions in your assignment. I have also explained this in the class
def laplace_prob(model, word, bigram, a, N): # word: string, bigram: tuple
  trigram = bigram + tuple((word,))
  return (tri_count(model, trigram) + a) / (bi_count(model, bigram) + (a * N))

# ---- Applying laplace smoothing to `model`----!!!!
def apply_laplace(model, a, N):
  for bigram, inner_dict in model.items():
    for word in list(inner_dict.keys()):
      new_prob = laplace_prob(model, word, bigram, a, N)
      model[bigram][word] = new_prob

apply_laplace(model, ALPHA, num_tokens)
print_model(model)

def check_sum(model): #check that the sum of probs is 1
  total = 0
  for w1_w2 in model:
    for w3, probability in model[w1_w2].items():
      total += probability
  return total

# print("total sum of probs:", check_sum(model)) # TODO: check math here... -Noah

Streaming output truncated to the last 5000 lines.

Current: ('absurdity', ',')
Next word: mustering, Probability: 1.000

Current: (',', 'mustering')
Next word: all, Probability: 1.000

Current: ('mustering', 'all')
Next word: my, Probability: 1.000

Current: ('my', 'latent')
Next word: power, Probability: 1.000

Current: ('latent', 'power')
Next word: of, Probability: 1.000

Current: ('of', 'hypocrisy')
Next word: to, Probability: 1.000

Current: ('hypocrisy', 'to')
Next word: sound, Probability: 1.000

Current: ('to', 'sound')
Next word: convinced, Probability: 1.000

Current: ('sound', 'convinced')
Next word: ., Probability: 1.000

Current: ('convinced', '.')
Next word: </s>, Probability: 1.000

Current: ('an', 'intersection')
Next word: ,, Probability: 1.000

Current: ('intersection', ',')
Next word: turning, Probability: 1.000

Current: ('turning', 'right')
Next word: ,, Probability: 1.000

Current: (',', 'chuffing')
Next word: to, Probability: 1.000

Current: ('chuffing', 'to')
N

In [375]:
# TODO: Develop a function to predict the next word
def predict_next_word(w1, w2):
    # TODO: Implement the prediction logic
    # Example: return max(model[w1, w2], key=model[w1, w2].get)
    # pass
    if (w1, w2) in model:
      return max(model[w1, w2], key=model[w1, w2].get, default=0)
    else:
      return None

print(predict_next_word('johnson', '\'s'))


fat


In [376]:
class TrigramClassifier:
    def __init__(self, model, genres_model, labels):
        self.model = model
        self.genres_model = genres_model
        self.genre_labels = numpy.unique(labels)
    def smooth_model(self):
      apply_laplace(self.model, ALPHA, num_tokens)
    def add_trigram(self, trigram, genre):
      laplace_probability = laplace_prob(self.model, trigram[2], (trigram[0], trigram[1]), ALPHA, num_tokens)
      self.model[(trigram[0], trigram[1])][trigram[2]] += laplace_probability
      self.genres_model[trigram][genre] += 1
    def calculate_joint_probability(self, sequence, genre=None):
        joint_probability = 1.0
        predicted_genre = None
        predictions = defaultdict(lambda: 1.0)
        # print(predictions)
        for i in range(2, len(sequence)):
            trigram = (sequence[i - 2], sequence[i - 1], sequence[i])
            genre_distribution = self.genres_model[trigram]
            if not genre_distribution:
              laplace_probability = laplace_prob(self.model, trigram[2], (trigram[0], trigram[1]), ALPHA, num_tokens)
              self.model[(trigram[0], trigram[1])][sequence[i]] += laplace_probability
              apply_laplace(self.model, ALPHA, num_tokens)
              if genre:
                self.genres_model[trigram][genre] += 1
              continue

            predicted_genre = max(genre_distribution, key=genre_distribution.get)
            conditional_probability = genre_distribution[predicted_genre]
            predictions[predicted_genre] += conditional_probability
        prob = max(predictions, key= lambda i: predictions[i])
        return predictions[prob], prob

    def predict(self, sequence):
        joint_probability, predicted_genre = self.calculate_joint_probability(sequence)
        return predicted_genre
# Initialize classifier
trigram_classifier = TrigramClassifier(model, genres_model, genres)

In [377]:
# TODO: Split your dataset into training, validation and testing sets - Do random sampling without replacement to create these sets, use basic python and numpy and not sklearn or other lib
def split_dataset(sentences, genres, training_ratio, validation_ratio):
  data = list(zip(sentences, genres))
  #get total size of our daatsets sets using ratios
  total_size = len(data)
  training_size = int(training_ratio * total_size)
  validation_size = int(validation_ratio * total_size)
  testing_size = total_size - training_size - validation_size

  #shuffle sentences using random sampling and split sets using array slicing
  shuffled_sentences = random.sample(data, total_size)
  training_set = shuffled_sentences[:training_size]
  validation_set = shuffled_sentences[training_size: training_size + validation_size]
  testing_set = shuffled_sentences[training_size + validation_size:]

  training_sentences, training_genres = zip(*training_set)
  validation_sentences, validation_genres = zip(*validation_set)
  testing_sentences, testing_genres = zip(*testing_set)

  return (
      list(training_sentences), list(training_genres),
      list(validation_sentences), list(validation_genres),
      list(testing_sentences), list(testing_genres)
  )

  # return training_set, validation_set, testing_set
train_sentences, train_genres, val_sentences, val_genres, test_sentences, test_genres = split_dataset(processed_sentences, genres, 0.50, 0.20)

In [378]:

# TODO: Train your classifier and run predictions on the validation set per epoch
def train_classifier(classifier, sentences, genres, epochs):
  y_true = []
  y_pred = []

  for i in range(epochs):
    for sentence, genre in zip(val_sentences, val_genres):
      prediction = classifier.predict(sentence)
      y_true.append(genre)
      y_pred.append(prediction)
  return y_true, y_pred

y_true, y_pred = train_classifier(trigram_classifier, train_sentences, train_genres, 10)

# TODO: Evaluate the performance of your model (accuracy, precision, recall, F1-score) -- check model performance on train and validation only
def evaluate_classifier(y_true, y_pred, genres):

    correct_predictions = sum(1 for true, pred in zip(y_true, y_pred) if true == pred)
    total_predictions = len(y_true)

    accuracy = correct_predictions / total_predictions if total_predictions > 0 else 0

    genre_indexes = {genre: i for i, genre in enumerate(genres)}

    true_positives = [0] * len(genres)
    false_positives = [0] * len(genres)
    false_negatives = [0] * len(genres)

    for true, pred in zip(y_true, y_pred):
        true_index = genre_indexes[true]
        pred_index = genre_indexes[pred]

        if true == pred:
            true_positives[true_index] += 1
        else:
            false_positives[pred_index] += 1
            false_negatives[true_index] += 1

    precision = [true_positive / (true_positive + false_positive) if (true_positive + false_positive) > 0 else 0 for true_positive, false_positive in zip(true_positives, false_positives)]
    recall = [true_positive / (true_positive + false_negative) if (true_positive + false_negative) > 0 else 0 for true_positive, false_negative in zip(true_positives, false_negatives)]
    f1_score = [2 * (p * r) / (p + r) if (p + r) > 0 else 0 for p, r in zip(precision, recall)]

    return accuracy, precision, recall, f1_score

# Evaluate on the validation set
accuracy_val, precision_val, recall_val, f1_val = evaluate_classifier(y_true, y_pred, numpy.unique(genres))

print("Validation Accuracy:", accuracy_val)
print("Validation Precision:", precision_val)
print("Validation Recall:", recall_val)
print("Validation F1 Score:", f1_val)

Validation Accuracy: 0.9885
Validation Precision: [0.99609375, 0.9959595959595959, 0.9763975155279503, 1.0]
Validation Recall: [0.9788867562380038, 0.986, 0.9974619289340102, 0.9842931937172775]
Validation F1 Score: [0.9874152952565343, 0.9909547738693467, 0.9868173258003766, 0.9920844327176782]


In [379]:
# TODO: Evaluate performance on test set only Once
y_true, y_pred = train_classifier(trigram_classifier, test_sentences, test_genres, 10)

accuracy_test, precision_test, recall_test, f1_test = evaluate_classifier(y_true, y_pred, numpy.unique(genres))

print("Test Accuracy:", accuracy_test)
print("Test Precision:", precision_test)
print("Test Recall:", recall_test)
print("Test F1 Score:", f1_test)

Test Accuracy: 0.9885
Test Precision: [0.99609375, 0.9959595959595959, 0.9763975155279503, 1.0]
Test Recall: [0.9788867562380038, 0.986, 0.9974619289340102, 0.9842931937172775]
Test F1 Score: [0.9874152952565343, 0.9909547738693467, 0.9868173258003766, 0.9920844327176782]
